In [4]:

from MalardClient.MalardClient import MalardClient
from MalardClient.DataSet import DataSet
from MalardClient.BoundingBox import BoundingBox
from MalardClient.MaskFilter import MaskFilter


from osgeo import ogr, osr
from datetime import datetime

import numpy as np

minX=-300000 
maxX=-200000
minY=-2400000
maxY=-2300000
minT=1298394420
maxT=1298871335 

client = MalardClient()
ds = DataSet( "cryotempo", "GRIS_BaselineC_Q2", "greenland"  )

proj4 = client.getProjection(ds).proj4

shapeFile = "/data/puma1/scratch/cryotempo/masks/icesheets.shp"

drv = ogr.GetDriverByName('ESRI Shapefile') #We will load a shape file
ds_in = drv.Open(shapeFile)    #Get the contents of the shape file

lyr_in = ds_in.GetLayer(0)    #Get the shape file's first layer

#print(lyr_in.GetFeatureCount())
outdriver=ogr.GetDriverByName('MEMORY')
source=outdriver.CreateDataSource('memData')

#open the memory datasource with write access
tmp=outdriver.Open('memData',1)

# Create a Polygon from the extent tuple
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(minX,minY)
ring.AddPoint(minX, maxY)
ring.AddPoint(maxX, maxY)
ring.AddPoint(maxX, minY)
ring.AddPoint(minX,minY)
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)

outLayer = source.CreateLayer("filter")

# Add an ID field
idField = ogr.FieldDefn("id", ogr.OFTInteger)
outLayer.CreateField(idField)

# Create the feature and set values
featureDefn = outLayer.GetLayerDefn()
feature = ogr.Feature(featureDefn)
feature.SetGeometry(poly)
feature.SetField("id", 1)
outLayer.CreateFeature(feature)
feature = None

result_layer = source.CreateLayer("clippedmask",geom_type=ogr.wkbPolygon)

lyr_in.Clip( outLayer, result_layer  )
    
def check(x, y, minX, maxX, minY, maxY):
    #Create a point
    pt = ogr.Geometry(ogr.wkbPoint)
    pt.SetPoint_2D(0, x, y)

    #Set up a spatial filter such that the only features we see when we
    #loop through "lyr_in" are those which overlap the point defined above
    
    result_layer.SetSpatialFilter(pt)
    #Loop through the overlapped features and display the field of interest
    if result_layer.GetFeatureCount() > 0:
        return True
    else:
        return False
    
def applyMask( xSrs, ySrs, minX, maxX, minY, maxY ):
    withinMask = np.zeros( len(xSrs) )
    
    for i, xy in enumerate(zip(xSrs, ySrs)):
        x,y = xy
        withinMask[i] = 1 if check(x,y, minX, maxX, minY, maxY) else 0
        
    return withinMask


In [5]:
#minX, maxX, minY, maxY = lyr_in.GetExtent()

bb = BoundingBox(minX, maxX, minY, maxY, minT, maxT )

start =datetime.now()
resultInfo = client.executeQuery(ds, bb )
postQuery = datetime.now()
df = resultInfo.to_df
postDf = datetime.now()
mask = applyMask(df['x'],df['y'],minX,maxX,minY,maxY)
postMask = datetime.now()
df['withinMask'] = mask

print("Started With {} points. Finished with {}".format(len(df),df['withinMask'].sum()))

totalDiff = (postMask - start).total_seconds()
totalMask = (postMask - postDf).total_seconds()
totalDf = (postDf - postQuery).total_seconds()
totalQuery = (postQuery - start).total_seconds()

print("Masking took {} in total, {} to query, {} to build df, {} to mask".format(totalDiff, totalQuery, totalDf, totalMask))

Started With 212185 points. Finished with 193723.0
Masking took 9.033143 in total, 0.146627 to query, 0.108632 to build df, 8.777884 to mask


In [7]:
f = MaskFilter(p_shapeFile=shapeFile)
res = client.executeQuery(ds, bb, maskFilters=f )

print(len(res.to_df))

client.releaseCacheHandle(res.resultFileName)

193723


['File deleted successfully [/data/puma1/scratch/v2/malard/export/cryotempo_GRIS_BaselineC_Q2_-133019006.nc]']